In [2]:
import tensorflow as tf
from models import decoder, encoder, vae, classifier
import pandas as pd
import pathlib
from constants import *
from utils import data_loading

In [3]:
apes_info = pd.read_csv(APES_INFO_FILEPATH)
all_images_ids, train_ids, validation_ids, test_ids = data_loading.get_image_ids(apes_info, pathlib.Path(DATA_FILEPATH))

In [4]:
dataset = data_loading.load_full_dataset(DATA_FILEPATH, IMAGE_SIZE, all_images_ids)

y_train = data_loading.get_feature_dataset(apes_info, FEATURE_NAMES, "train")
y_validation = data_loading.get_feature_dataset(apes_info, FEATURE_NAMES, "validation")
y_test = data_loading.get_feature_dataset(apes_info, FEATURE_NAMES, "test")

Found 10000 files belonging to 10000 classes.


In [5]:
decoder_model = decoder.build_decoder(LATENT_DIM)
encoder_model = encoder.build_encoder(LATENT_DIM)

vae_model = vae.VAE(encoder_model, decoder_model, RECONSTRUCTION_LOSS_WEIGHT, KL_LOSS_WEIGHT)
vae_model.load_weights(MODEL_VAE_FILEPATH)

classifier_model = classifier.build_classifier(encoder_model, N_UNIQUE_FEATURES, FEATURE_NAMES)
classifier_model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(),
    loss=[tf.keras.losses.BinaryCrossentropy(from_logits=False)] * len(FEATURE_NAMES),
    metrics=["accuracy"],
)

In [6]:
x_train = data_loading.load_specific_dataset(dataset, train_ids, None)
train_dataset = tf.data.Dataset.zip((x_train, y_train)).batch(BATCH_SIZE)

x_validation = data_loading.load_specific_dataset(dataset, validation_ids, None)
validation_dataset = tf.data.Dataset.zip((x_validation, y_validation)).batch(BATCH_SIZE)

In [7]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_CLASSIFIER_FILEPATH,
    save_weights_only=True,
    monitor="loss",
    mode="min",
    save_best_only=True,
)

csv_logger = tf.keras.callbacks.CSVLogger(HISTORY_CLASSIFIER_FILEPATH, append=True)

In [9]:
history = classifier_model.fit(
    train_dataset.repeat(STEPS_PER_EPOCH * EPOCHS_CLASSIFIER),
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=10,
    batch_size=None,
    validation_data=validation_dataset,
)

Epoch 1/10


2023-05-23 22:43:15.778423: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype bool and shape [7000,44]
	 [[{{node Placeholder/_20}}]]
2023-05-23 22:43:15.778642: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype bool and shape [7000,44]
	 [[{{node Placeholder/_20}}]]


54/54 [==============================] - ETA: 0s - loss: 1.7255 - Mouth_loss: 0.1736 - Background_loss: 0.3897 - Hat_loss: 0.1862 - Eyes_loss: 0.2322 - Clothes_loss: 0.1868 - Fur_loss: 0.2496 - Earring_loss: 0.3074 - Mouth_accuracy: 0.1505 - Background_accuracy: 0.2815 - Hat_accuracy: 0.1616 - Eyes_accuracy: 0.1059 - Clothes_accuracy: 0.1262 - Fur_accuracy: 0.1040 - Earring_accuracy: 0.6740

2023-05-23 22:43:43.700652: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype bool and shape [1500,8]
	 [[{{node Placeholder/_14}}]]
2023-05-23 22:43:43.700830: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10000]
	 [[{{node Placeholder/_0}}]]


54/54 [==============================] - 43s 779ms/step - loss: 1.7255 - Mouth_loss: 0.1736 - Background_loss: 0.3897 - Hat_loss: 0.1862 - Eyes_loss: 0.2322 - Clothes_loss: 0.1868 - Fur_loss: 0.2496 - Earring_loss: 0.3074 - Mouth_accuracy: 0.1505 - Background_accuracy: 0.2815 - Hat_accuracy: 0.1616 - Eyes_accuracy: 0.1059 - Clothes_accuracy: 0.1262 - Fur_accuracy: 0.1040 - Earring_accuracy: 0.6740 - val_loss: 1.3228 - val_Mouth_loss: 0.1385 - val_Background_loss: 0.2335 - val_Hat_loss: 0.1391 - val_Eyes_loss: 0.1973 - val_Clothes_loss: 0.1259 - val_Fur_loss: 0.2201 - val_Earring_loss: 0.2684 - val_Mouth_accuracy: 0.2080 - val_Background_accuracy: 0.8207 - val_Hat_accuracy: 0.2147 - val_Eyes_accuracy: 0.1660 - val_Clothes_accuracy: 0.1807 - val_Fur_accuracy: 0.1407 - val_Earring_accuracy: 0.7147
Epoch 2/10
54/54 [==============================] - 42s 789ms/step - loss: 1.1819 - Mouth_loss: 0.1240 - Background_loss: 0.1742 - Hat_loss: 0.1217 - Eyes_loss: 0.1802 - Clothes_loss: 0.1089 - F